In [7]:
# ***********************************
# Filename: 분석보고서_모수_차집합_통계_산출.py 
# Author: Haeun Jeong
# 오라클에 적재된 데이터를 기준으로 예전과 현재를 비교해 체납한 사람을 도출
# 
# **********************************


# **********************************
# Module Importation
# **********************************

import cx_Oracle
import os
import csv
from datetime import date
import glob
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np


# **********************************
# Setting OS ENV
# **********************************

os.putenv('NLS_LANG', 'KOREAN_KOREA.KO16MSWIN949')


# **********************************
# Global Constant
# **********************************

UID = 'sibum'
PW = 'imsi00.!'
HOST_NAME = '10.46.107.124:22751/SIBUMDB'


# **********************************
# Global Variable
# **********************************


today = "20200210" # 정보를 불러올 날짜 예) 20200810
generate_date = date.today().strftime("%Y_%m_%d") # 파일 이름에 쓰일 오늘 날짜
file_name = f"{today}_기준_분석보고서_세목_통계_산출_generated_{generate_date}.xlsx" # 엑셀 파일 이름

has_error = False

table_name = "HJ_REPORT_INFO"

df = ""
df_score = ""
# **********************************
# Define Class
# **********************************

class Oracle:
    uid = UID
    pw = PW
    host_name = HOST_NAME
    connection = None
    
    def __init__(self): # 생성 시 DB 연결 및 
        self.connect_db()

        
    def __del__(self): # 소멸 시 DB 연결 해제
        return self.close_db()

    
    def connect_db(self):
        self.connection = cx_Oracle.connect(self.uid, self.pw, self.host_name)
        self.cursor = self.connection.cursor()
        return self.cursor
    
    
    def reconnect_db(self):
        self.close_db()
        return self.connect_db
    
    
    def close_db(self):
        if self.connection != None:
            self.connection.close()
            return 0
        else:
            return 1
    
    def execute_query(self, query, check_error = False):
        try:
            self.cursor.execute(query)
#             print("성공")
            return False
        except Exception as e:
            print(query)
            print(f"에러가 발생했습니다. {str(e)}")
            if check_error:
                return True
    
    
    def print_query_result(self):
        self.cursor.rowfactory = make_dict_factory(self.cursor)
        self.query_results = self.cursor.fetchall()
        for query in self.query_results:
            print(query)


# **********************************
# Entry Point
# **********************************

if __name__ == "__main__":                 
        
        print("-- 프로그램 시작 --")
        oracle = Oracle()
        
        # 등급, 세목별 enc 개수 구하기
        query = f"""SELECT bug.cn_semok, COUNT(DISTINCT rep.tpr_no_enc), rep.r_fin_gdr
                    FROM a01_bug bug, hj_report_info rep
                    WHERE rep.tpr_no_enc = bug.tpr_no_enc AND rep.today = {today} AND rep.model_grp != '미산출대상'
                    GROUP BY bug.cn_semok, rep.r_fin_gdr, model_grp
                    ORDER BY bug.cn_semok asc"""
        df = pd.read_sql(query, con=oracle.connection)
        df_pivot = pd.pivot_table(df, values='COUNT(DISTINCTREP.TPR_NO_ENC)', index = ['R_FIN_GDR'], columns = ['CN_SEMOK'], aggfunc=np.sum)
        df_pivot = df_pivot.replace(np.nan, '', regex = True)
        display(df_pivot)

        if os.path.exists(f"./{file_name}"): # 해당 날짜의 엑셀 파일이 기존에 있으면 해당 엑셀파일을 지운다.
            os.remove(file_name)
        
        with pd.ExcelWriter(file_name)as writer:
            df_pivot.to_excel(writer, sheet_name = f"{today}기준_세목_등급")
        print("-- 프로그램 종료 --")


-- 프로그램 시작 --


CN_SEMOK,구)재산세(건축물),구)재산세(주택),구)재산세(토지),구)취득세(기계장비),구)취득세(부동산),구)취득세(선박),구)취득세(차량),담배소비세,등록면허세(등록),등록면허세(면허),...,지방소득세(종합소득),지방소득세(특별징수),지역자원시설세(특자),취득세(기계장비),취득세(기타),취득세(부동산),취득세(선박),취득세(이륜차량),취득세(차량),취득세(항공기)
R_FIN_GDR,,,,,,,,,,,,,,,,,,,,,
1,,,2.0,,3.0,,,2.0,16686.0,6716.0,...,13747.0,3223.0,79.0,204.0,39.0,9456.0,32.0,690.0,17926.0,
2,9,,4.0,,23.0,,8,6.0,134224.0,93565.0,...,125109.0,34888.0,914.0,1862.0,265.0,66118.0,282.0,6928.0,154567.0,
3,3,6,13.0,1,44.0,1,8,19.0,257476.0,162182.0,...,247258.0,48017.0,967.0,3324.0,443.0,99644.0,401.0,15555.0,264596.0,3
4,2,1,5.0,,5.0,2,3,2.0,60391.0,25502.0,...,67449.0,6427.0,184.0,614.0,119.0,9697.0,71.0,6373.0,45448.0,
5,,3,2.0,,3.0,,1,1.0,10450.0,5855.0,...,10997.0,1279.0,48.0,64.0,27.0,1180.0,1.0,782.0,5324.0,1


-- 프로그램 종료 --
